In [ ]:
!pip install tensorflow

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.optimizers import Adam

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import sys

import matplotlib.pyplot as plt

In [2]:
#folder_repository = "/Users/harun/Desktop/Uni/4. Semester/Data Mining/DataMining_TeamA"
folder_repository = "C:/Users/andre/Documents/Github/DataMining_TeamA"

folder_data = f"{folder_repository}/resources/data_classification"
folder_labels = f"{folder_repository}/resources/data_classification/labels"
folder_output = f"{folder_repository}/output/classification"


In [3]:
data = pd.read_csv(f"{folder_data}/x2.csv")
data_labels = pd.read_json(f"{folder_data}/labels/y2.json")

labels = []
# add labels to dataset
for i in range(0, len(data_labels)):
    labels.append(data_labels.iloc[i].item())

data.iloc[:, -1] = labels
data.to_csv('x2.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/andre/Documents/Github/DataMining_TeamA/resources/data_classification/x2.csv'

In [4]:
def accuracy(Y_test, predictions, dataset):
    # extract neurons with most activation
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(np.argmax(prediction))

    accuracy = accuracy_score(Y_test, predicted_classes)
    print(f"{dataset};{accuracy};", end="")


In [ ]:
def feature_importance(model):
    # weights of the model
    weights = model.get_weights()

    # weights of first layer do have the most expressiveness, because it is directly connected to the input data
    weights_first_layer = weights[0]

    # number of features
    num_features = weights_first_layer.shape[0]

    # generate feature names
    feature_names = [f"Feature {i+1}" for i in range(num_features)]

    # compute sum of input layer
    feature_importance = np.sum(np.abs(weights_first_layer), axis=1)

    # sort list with decreasing importance
    sorted_features = sorted(zip(feature_importance, feature_names), reverse=True)

    #print(len(weights_first_layer[0]))
    # print
    for importance, feature_name in sorted_features:
        print(f"{feature_name}:, Importance: {importance / 64}")

In [ ]:
def model_arch():
    datasets = ['x0_with_labels.csv', 'x1_with_labels.csv', 'x2_with_labels.csv']
    for dataset in datasets:
        data = pd.read_csv(f"{folder_data}/{dataset}")

        # Split datasets into train and test
        X_train, X_test, Y_train, Y_test = train_test_split(data.iloc[:, :-1], data.iloc[:, -1], test_size=0.25, shuffle = True)

        # labels are intepretated as ordinal, so the model need to know, that the labels are categories
        Y_train_encoded = to_categorical(Y_train)
        Y_test_encoded = to_categorical(Y_test)

        # Create NN model
        model = Sequential()
        model.add(Dense(64, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(30, activation='softmax'))
        
        optimizer=Adam(lr=0.001)

        # compile model
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

        # train model
        model.fit(X_train, Y_train_encoded, epochs=10, batch_size=32)
        
        predictions = model.predict(X_test)
        accuracy(Y_test, predictions, dataset[:2])
        #feature_importance(model)
model_arch()

In [5]:
def model_arch(testsplit: float, activation, hidden_layers: int , neurons: int, learn_rate_adam: float, epochs: int, batch_size: int, metrics: list):
    datasets = ['x0_with_labels.csv', 'x1_with_labels.csv', 'x2_with_labels.csv']
    for dataset in datasets:
        data = pd.read_csv(f"{folder_data}/{dataset}")

        # Split datasets into train and test
        X_train, X_test, Y_train, Y_test = train_test_split(data.iloc[:, :-1], data.iloc[:, -1], test_size=testsplit, shuffle = True)

        # labels are intepretated as ordinal, so the model need to know, that the labels are categories
        Y_train_encoded = to_categorical(Y_train)
        Y_test_encoded = to_categorical(Y_test)

        # Create NN model
        model = Sequential()
        
        for x in range(hidden_layers):
             model.add(Dense(neurons, activation=activation))
        model.add(Dense(30, activation='softmax'))
        
        optimizer=Adam(learning_rate=learn_rate_adam)

        # compile model
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=metrics)

        # train model
        model.fit(X_train, Y_train_encoded, epochs=epochs, batch_size=batch_size)
        
        predictions = model.predict(X_test)
        accuracy(Y_test, predictions, dataset[:2])
        #feature_importance(model)
        

In [6]:
sys.stdout.close()
sys.stdout = sys.__stdout__
print("hallo")

In [ ]:
filename=f"{folder_output}/acc2.csv"
mode="w"
sys.stdout = open(filename, mode)

print(f"{t};{a};{h};{n};{l};{e};{b};{m};", end="")
model_arch(testsplit=0.25, activation='relu', hidden_layers=2 , neurons=64, learn_rate_adam=0.001, epochs=10, batch_size=30, metrics=['accuracy'])
print("0")

In [ ]:
filename=f"{folder_output}/acc2.csv"
mode="w"
sys.stdout = open(filename, mode)


testsplit=[0.1, 0.25, 0.5] #[0.1, 0.15, 0.2, 0.25, 0.3, 0.5]
activation=['relu', 'sigmoid', 'linear'] #['relu', 'sigmoid', 'tanh', 'linear']
hidden_layers=[2, 3, 4, 5]
neurons=[32, 64, 128]
learn_rate_adam=[0.0001]
epochs=[4, 8, 12, 16, 20]
batch_size=[32, 64, 128]
metrics=[['accuracy'], ['top_k_accuracy']] #[['accuracy'], ['top_k_accuracy'], ['true_positives']]

print(f"testsplit;activation;hidden_layers;neurons;learn_rate_adam;epochs;batch_size;metrics;x0;acc_x0;x1;acc_x1;x2;acc_x2;0")

for t in testsplit:
    for a in activation:
        for h in hidden_layers:
            for n in neurons:
                for l in learn_rate_adam:
                    for e in epochs:
                        for b in batch_size:
                            for m in metrics:
                                print(f"{t};{a};{h};{n};{l};{e};{b};{m};", end="")
                                model_arch(testsplit=t, activation=a, hidden_layers=h , neurons=n, learn_rate_adam=l, epochs=e, batch_size=b, metrics=m)
                                print("0")
